## Introduction
Greetings from the Kaggle bot! This is an automatically-generated kernel with starter code demonstrating how to read in the data and begin exploring. If you're inspired to dig deeper, click the blue "Fork Notebook" button at the top of this kernel to begin editing.

## Exploratory Analysis
To begin this exploratory analysis, first import libraries and define functions for plotting the data using `matplotlib`. Depending on the data, not all plots will be made. (Hey, I'm just a simple kerneling bot, not a Kaggle Competitions Grandmaster!)

In [ ]:
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import hvplot.pandas  # noqa
import json
import ast
import random
hvplot.extension("bokeh")

: 

There is 1 csv file in the current version of the dataset:


In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


: 

The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [ ]:
def uniques(duplicates): # From: https://www.reddit.com/r/Python/comments/jfx7wy/how_to_quickly_remove_duplicates_from_a_list/
    unique = []
    seen = set()
    for element in duplicates:
        if element not in seen:
            unique.append(element)
            seen.add(element)
    return unique

: 

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


: 

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


: 

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


: 

Now you're ready to read in the data and use the plotting functions to visualize the data.

### Let's check 1st file: /kaggle/input/Food Ingredients and Recipe Dataset with Image Name Mapping.csv

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# Food Ingredients and Recipe Dataset with Image Name Mapping.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('Epicurious Recepies Kagle.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'Epicurious Recepies Kagle'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

: 

Let's take a quick look at what the data looks like:

In [ ]:
df1.hvplot.table(height=600,fit_columns=True,width=2000)

: 

In [ ]:
df1.dtypes

: 

In [ ]:
parsed_ingredients = []
parsed_diet_restrictions = []
with open("recipe_food_items.json", "r") as fh:
    for line in fh.readlines():
        line = line.rstrip("\n").rstrip(",")
        result = json.loads(line)
        output = []
        for ingredient in result.get("ingredients",[]):
            parsed = ingredient.get("parsed", [])
            for parsed_ingredient in parsed:
                food = parsed_ingredient.get("food", None)
                if food is None:
                    continue
                output += [food]
        parsed_diet_restrictions.append(result.get("dietLabels",[]) + result.get("healthLabels",[]))
        parsed_ingredients.append(output)
# print(parsed_diet_restrictions)

: 

In [ ]:
all_ingredients = []

for ingredient_list_str in parsed_ingredients:
    all_ingredients += ingredient_list_str

all_ingredients = uniques(all_ingredients)
# print(all_ingredients)

: 

In [ ]:
food_details = {}
with open("foods.json", "r") as fh:
    food_details = json.loads(fh.read())

def get_human_readable_dietary_name(name):
    return name.lower().replace("_", " ")

get_human_readable_dietary_name(food_details["chicken"]["dietLabels"][0])

: 

In [ ]:
def create_training_prompt_pair(title, ingredients, pure_ingredients, directions, diet_modifiers, num_required_ingredients_in_pantry = 4, num_extra_listed_pantry_ingredients = 3):
    prompt = "Food in my pantry:\n"

    num_required_ingredients_in_pantry = min(len(pure_ingredients),num_required_ingredients_in_pantry)
    pantry_ingredients = uniques(random.sample(pure_ingredients, num_required_ingredients_in_pantry))
    while len(pantry_ingredients) < num_required_ingredients_in_pantry + num_extra_listed_pantry_ingredients:
        pantry_ingredients += random.sample(all_ingredients, num_extra_listed_pantry_ingredients)
        pantry_ingredients = uniques(pantry_ingredients)

    diet_modifiers = set(diet_modifiers)

    for pure_food in pantry_ingredients:
        food_restrictions = set(food_details[pure_food].get("dietLabels",[]))
        # print(food_restrictions)
        diet_modifiers = diet_modifiers.intersection(food_restrictions)

    prompt += ", ".join([ p.strip() for p in pantry_ingredients]) + "\n\n"

    if len(diet_modifiers) > 0:
        prompt +=  "Keywords:\n" + ", ".join([get_human_readable_dietary_name(diet) for diet in list(diet_modifiers)]) + "\n\n"

    # prompt +=  "Recipe Search: " + " ".join(random.sample(title,3)) + "\n\n"

    response = title + "\n\n"

    response += "Ingredients:\n"
    response += "\n".join(ingredients)

    response += "\n\nDirections:\n"
    response += directions

    return (prompt, response)

# diet_modifiers = ["Vegetarian", "Vegan", "Paleo", "High Fiber", "High Protein", "Low Carb", "Low Fat", "Low Sodium", "Low Sugar", "Alcohol Free", "Balanced"]
# alergen_modifiers = ["Gluten Free","Dairy Free","Egg Free","Soy Free","Wheat Free","Fish Free","Shellfish Free","Tree Nut Free","Peanut Free"]

def get_prompt_from_dataset(index = 0):
    row = df1.iloc[index]
    pure_ingredients = parsed_ingredients[index]
    raw_diet_restrictions = parsed_diet_restrictions[index]


    # print(parsed_ingredients)
    num_diet_restrictions = random.randint(0, len(raw_diet_restrictions))
    diet_restrictions = random.sample(raw_diet_restrictions, num_diet_restrictions)


    title = str(row["Title"])
    instructions = str(row["Instructions"])
    cleaned_ingredients = ast.literal_eval(str(row["Cleaned_Ingredients"]))
    num_required_ingredients = len(cleaned_ingredients)
    num_required_ingredients_in_pantry_u = random.randint(min(3,num_required_ingredients),num_required_ingredients)

    (prompt,response) = create_training_prompt_pair(
        title,
        cleaned_ingredients,
        pure_ingredients,
        instructions,
        diet_restrictions,
        num_required_ingredients_in_pantry=num_required_ingredients_in_pantry_u,
        num_extra_listed_pantry_ingredients=random.randint(1,10)
    ) #num_required_ingredients_in_pantry=num_required_ingredients_in_pantry)

    return (prompt, response)

(p,r) = get_prompt_from_dataset(0)

print("PROMPT:")
print(p)

print("\nDESIRED RESPONSE:")
print(r)

: 

In [ ]:
# # For GPT-3 training
# with open("training_data.json", "w") as fh:
#     for i in range(df1.shape[0]):
#         for v in range(3):
#             (prompt, response) = get_prompt_from_dataset(i)
#             fh.write(json.dumps({"prompt": prompt, "completion": response},ensure_ascii=False) + "\n")
#         fh.flush()

: 

In [ ]:
# for CO:HERE training
with open("cohere_training_data.txt", "w") as fh:
    for i in range(df1.shape[0]):
        for v in range(3):
            (prompt, response) = get_prompt_from_dataset(i)
            fh.write(prompt + "Recipe:\n" +response  + "END\n--S--\n")
        fh.flush()

: 

In [ ]:
import time
import requests

num_completed = 0
try:
    with open("recipe_food_items.json", "r") as fh:
        a = "[" + fh.read().rstrip("\n").rstrip(",") + "]"
        num_completed = len(json.loads(a))
except FileNotFoundError:
    pass

print("num_completed", num_completed)

with open("recipe_food_items.json", "a") as fh:
    for i in range(num_completed, df1.shape[0]):
        ingredients = df1.loc[i, 'Cleaned_Ingredients']
        request_body = json.dumps({"ingr":ast.literal_eval(str(ingredients))})
        r = requests.post("https://api.edamam.com/api/nutrition-details?app_id=47379841&app_key=d28718060b8adfd39783ead254df7f92", data=request_body, headers={'content-type': 'application/json'})
        try:
            result = json.loads(r.text)
            output = {}
            output['dietLabels'] = result.get("dietLabels",None)
            output['healthLabels'] = result.get("healthLabels",None)
            output["cautions"] = result.get("cautions",None)
            output["ingredients"] = []
            for ingredient in result.get("ingredients",[]):
                parsed = ingredient.get("parsed", [])
                parsed_out = []
                for parsed_ingredient in parsed:
                    parsed_out.append({
                        "quantity": parsed_ingredient.get("quantity", None),
                        "measure": parsed_ingredient.get("measure", None),
                        "foodMatch": parsed_ingredient.get("foodMatch", None),
                        "food": parsed_ingredient.get("food", None),
                        "foodId": parsed_ingredient.get("foodId", None)
                    })
                output["ingredients"].append({
                    "text": ingredient.get("text", None),
                    "parsed": parsed_out
                })


            print("Completed #", i)
            fh.write(json.dumps(output) + ",\n")
            time.sleep(1)

        except json.decoder.JSONDecodeError:
            print("Got Invalid JSON", r.text)
            break
        if result.get('ingredients',None) is None:
            print("Got Wrong Response", r.text, result)
            break



# for i in range():

# "https://api.edamam.com/api/nutrition-details?app_id=47379841&app_key=d28718060b8adfd39783ead254df7f92", {
#     "referrer": "https://developer.edamam.com/",
#     "body": "{\"ingr\":[\"¼ cup hazelnuts\",\"1 pound bow tie pasta (farfalle)\",\"8 tablespoons unsalted butter, plus more if needed\",\"4 cloves of garlic, minced\",\"Hefty pinch each of salt and freshly ground black pepper\",\"Small pinch of crushed red pepper flakes\",\"1 bunch Swiss chard, stems finely chopped and greens thinly sliced\",\"4 ounces Parmesan cheese, shaved\",\"2 tablespoons balsamic vinegar (optional)\"]}",
#     "method": "POST",
#     "mode": "cors"
# }

: 

In [ ]:
def parse_user_input(input_string):
    input_words = input_string.split()
    request_body = json.dumps({"ingr":input_words})
    r = requests.post("https://api.edamam.com/api/nutrition-details?app_id=47379841&app_key=d28718060b8adfd39783ead254df7f92", data=request_body, headers={'content-type': 'application/json'})
    try:
        result = json.loads(r.text)
        if result.get('ingredients',None) is None:
            print("Got Wrong Response", r.text, result)
            return []
        output = []
        for ingredient in result.get("ingredients",[]):
            parsed = ingredient.get("parsed", [])
            for parsed_ingredient in parsed:
                pure_ingredient = parsed_ingredient.get("food", None)
                if pure_ingredient is not None:
                    output.append(pure_ingredient)


        return output

    except json.decoder.JSONDecodeError:
        print("Got Invalid JSON", r.text)
        return []


: 

In [ ]:
parse_user_input("I got roast turkey and half a cup of coffee. Also got creamer.")

: 

In [ ]:
# diet_modifiers = ["Vegetarian", "Vegan", "Paleo", "High Fiber", "High Protein", "Low Carb", "Low Fat", "Low Sodium", "Low Sugar", "Alcohol Free", "Balanced"]
# alergen_modifiers = ["Gluten Free","Dairy Free","Egg Free","Soy Free","Wheat Free","Fish Free","Shellfish Free","Tree Nut Free","Peanut Free"]

diet_modifiers = ["Vegetarian", "Vegan", "Paleo", "High Fiber", "High Protein", "Low Carb", "Low Fat", "Low Sodium", "Low Sugar", "Alcohol Free", "Balanced"]
alergen_modifiers = ["Gluten Free","Dairy Free","Egg Free","Soy Free","Wheat Free","Fish Free","Shellfish Free","Tree Nut Free","Peanut Free"]

diet_modifiers = ["Vegetarian", "Vegan", "Paleo", "High Fiber", "High Protein", "Low Carb", "Low Fat", "Low Sodium", "Low Sugar", "Alcohol Free", "Balanced"]
alergen_modifiers = ["Gluten Free","Dairy Free","Egg Free","Soy Free","Wheat Free","Fish Free","Shellfish Free","Tree Nut Free","Peanut Free"]

: 

In [ ]:
foods = {}

with open("recipe_food_items.json", "r") as fh:
    for line in fh.readlines():
        line = line.rstrip("\n").rstrip(",")
        result = json.loads(line)

        diet_labels = set(result.get("dietLabels",[]) + result.get("healthLabels",[]))
        # print(diet_labels)
        for ingredient in result.get("ingredients",[]):
            parsed = ingredient.get("parsed", [])
            for parsed_ingredient in parsed:
                food = parsed_ingredient.get("food", None)
                foodLabel = parsed_ingredient.get("foodMatch", None)
                if food is None:
                    continue
                if food not in foods:
                    foodLabels = set()
                    foodLabels.add(food)
                    foods[food] = {
                        "dietLabels": set().union(diet_labels),
                        "foodLabelAsFound": foodLabels
                    }
                else:
                    foods[food]["dietLabels"] = foods[food]["dietLabels"].union(diet_labels)
                    foods[food]["foodLabelAsFound"].add(food)

for food in foods:
    foods[food]["dietLabels"] = list(foods[food]["dietLabels"])
    foods[food]["foodLabelAsFound"] = list(foods[food]["foodLabelAsFound"])

with open("foods.json", "w") as fh:
    json.dump(foods, fh)


: 

: 